In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
drop_df = application_df.drop(['EIN','NAME'], axis=1)
drop_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
drop_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
drop_df.value_counts("APPLICATION_TYPE")


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff_value = 100

# use the variable name `application_types_to_replace`
application_types_to_replace = []

# Find application types occurring less frequently than the cutoff value
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_counts[application_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    drop_df['APPLICATION_TYPE'] = drop_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
drop_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
drop_df.value_counts("CLASSIFICATION")

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C2380        1
C2500        1
C2561        1
C8210        1
Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
drop_df.value_counts("CLASSIFICATION") >1

CLASSIFICATION
C1000     True
C2000     True
C1200     True
C3000     True
C2100     True
         ...  
C2190    False
C2380    False
C2500    False
C2561    False
C8210    False
Length: 71, dtype: bool

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
cutoff_value = 100
# use the variable name `classifications_to_replace`
classifications_to_replace = []
#  YOUR CODE GOES HERE
# Find application types occurring less frequently than the cutoff value
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    drop_df['CLASSIFICATION'] = drop_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
drop_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
preprocessed_data = pd.get_dummies(drop_df)
preprocessed_data

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
#Check out column names
preprocessed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 51 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ASK_AMT                       34299 non-null  int64
 2   IS_SUCCESSFUL                 34299 non-null  int64
 3   APPLICATION_TYPE_Other        34299 non-null  uint8
 4   APPLICATION_TYPE_T10          34299 non-null  uint8
 5   APPLICATION_TYPE_T19          34299 non-null  uint8
 6   APPLICATION_TYPE_T3           34299 non-null  uint8
 7   APPLICATION_TYPE_T4           34299 non-null  uint8
 8   APPLICATION_TYPE_T5           34299 non-null  uint8
 9   APPLICATION_TYPE_T6           34299 non-null  uint8
 10  APPLICATION_TYPE_T7           34299 non-null  uint8
 11  APPLICATION_TYPE_T8           34299 non-null  uint8
 12  APPLICATION_TYPE_T9           34299 non-null  uint8
 13  AFFILIATION_CompanySponsored  3

In [11]:
# Select the features (X)
X = preprocessed_data[['STATUS', 'ASK_AMT', 'APPLICATION_TYPE_Other', 'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6', 'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8', 'APPLICATION_TYPE_T9', 'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent', 'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other', 'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200', 'CLASSIFICATION_C1270', 'CLASSIFICATION_C1700', 'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C2700', 'CLASSIFICATION_C3000', 'CLASSIFICATION_C4000', 'CLASSIFICATION_C5000', 'CLASSIFICATION_C7000', 'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare', 'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev', 'ORGANIZATION_Association', 'ORGANIZATION_Co-operative', 'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0', 'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999', 'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M', 'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M', 'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y']]

# Select the target variable (y)
y = preprocessed_data['IS_SUCCESSFUL']

# Print the shapes of X and y to verify the split
print('Shape of features (X):', X.shape)
print('Shape of target (y):', y.shape)


Shape of features (X): (34299, 50)
Shape of target (y): (34299,)


In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [14]:
#Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
layer = tf.keras.layers.Dense(units=5, activation="relu", input_dim=50)
nn_model.add(layer)


# Second hidden layer
layer2 = tf.keras.layers.Dense(units=10, activation="relu")
nn_model.add(layer2)

# Add the third Dense hidden layer with 3 units and ReLU activation
layer3 = tf.keras.layers.Dense(units=3, activation="relu")
nn_model.add(layer3)

# Output layer
output_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")
nn_model.add(output_layer)

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 255       
                                                                 
 dense_1 (Dense)             (None, 10)                60        
                                                                 
 dense_2 (Dense)             (None, 3)                 33        
                                                                 
 dense_3 (Dense)             (None, 1)                 4         
                                                                 
Total params: 352
Trainable params: 352
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
layer = tf.keras.layers.Dense(units=5, activation="relu", input_dim=50)
nn_model.add(layer)


# Second hidden layer
layer2 = tf.keras.layers.Dense(units=10, activation="relu")
nn_model.add(layer2)

# Add the third Dense hidden layer with 3 units and ReLU activation
layer3 = tf.keras.layers.Dense(units=3, activation="relu")
nn_model.add(layer3)

# Output layer
output_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")
nn_model.add(output_layer)

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 5)                 255       
                                                                 
 dense_5 (Dense)             (None, 10)                60        
                                                                 
 dense_6 (Dense)             (None, 3)                 33        
                                                                 
 dense_7 (Dense)             (None, 1)                 4         
                                                                 
Total params: 352
Trainable params: 352
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=20, verbose=1)

Epoch 1/20
858/858 [==============================] - 5s 2ms/step - loss: 0.6122 - accuracy: 0.6819
Epoch 2/20
858/858 [==============================] - 1s 2ms/step - loss: 0.5678 - accuracy: 0.7182
Epoch 3/20
858/858 [==============================] - 1s 2ms/step - loss: 0.5609 - accuracy: 0.7212
Epoch 4/20
858/858 [==============================] - 1s 2ms/step - loss: 0.5562 - accuracy: 0.7251
Epoch 5/20
858/858 [==============================] - 1s 2ms/step - loss: 0.5538 - accuracy: 0.7280
Epoch 6/20
858/858 [==============================] - 1s 2ms/step - loss: 0.5526 - accuracy: 0.7291
Epoch 7/20
858/858 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7305
Epoch 8/20
858/858 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7303
Epoch 9/20
858/858 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7301
Epoch 10/20
858/858 [==============================] - 1s 2ms/step - loss: 0.5502 - accuracy: 0.7300

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5575 - accuracy: 0.7261 - 945ms/epoch - 4ms/step
Loss: 0.5574941635131836, Accuracy: 0.7260932922363281


In [19]:
# Define the model
model = tf.keras.models.Sequential()

# Add the input layer
model.add(tf.keras.layers.Dense(units=10, activation='relu', input_dim=50))

# Add a dropout layer after the first hidden layer
model.add(tf.keras.layers.Dropout(rate=0.2))

# Add the second hidden layer
model.add(tf.keras.layers.Dense(units=5, activation='relu'))

# Add a dropout layer after the second hidden layer
model.add(tf.keras.layers.Dropout(rate=0.2))

# Add the output layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
858/858 [==============================] - 4s 3ms/step - loss: 8440.2490 - accuracy: 0.4957
Epoch 2/10
858/858 [==============================] - 2s 2ms/step - loss: 4603.4741 - accuracy: 0.4870
Epoch 3/10
858/858 [==============================] - 1s 2ms/step - loss: 1108.5209 - accuracy: 0.4989
Epoch 4/10
858/858 [==============================] - 1s 2ms/step - loss: 1108.3396 - accuracy: 0.5096
Epoch 5/10
858/858 [==============================] - 1s 2ms/step - loss: 414.2615 - accuracy: 0.5149
Epoch 6/10
858/858 [==============================] - 1s 2ms/step - loss: 393.4087 - accuracy: 0.5243
Epoch 7/10
858/858 [==============================] - 2s 2ms/step - loss: 94.1444 - accuracy: 0.5024
Epoch 8/10
858/858 [==============================] - 1s 2ms/step - loss: 40.9672 - accuracy: 0.5214
Epoch 9/10
858/858 [==============================] - 2s 2ms/step - loss: 3.4917 - accuracy: 0.5314
Epoch 10/10
858/858 [==============================] - 2s 2ms/step - loss: 0.6918 

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5575 - accuracy: 0.7261 - 261ms/epoch - 1ms/step
Loss: 0.5574941635131836, Accuracy: 0.7260932922363281


In [21]:
# Define the model
model = tf.keras.models.Sequential()

# Add the input layer
model.add(tf.keras.layers.Dense(units=10, activation='relu', input_dim=50))

# Add a dropout layer after the first hidden layer
model.add(tf.keras.layers.Dropout(rate=0.2))

# Add the second hidden layer
model.add(tf.keras.layers.Dense(units=5, activation='relu'))

# Add a dropout layer after the second hidden layer
model.add(tf.keras.layers.Dropout(rate=0.2))

# Add the third hidden layer
model.add(tf.keras.layers.Dense(units=3, activation='relu'))

# Add the output layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=5)

Epoch 1/10
5488/5488 [==============================] - 12s 2ms/step - loss: 3148.4282 - accuracy: 0.5126
Epoch 2/10
5488/5488 [==============================] - 10s 2ms/step - loss: 163.1556 - accuracy: 0.5240
Epoch 3/10
5488/5488 [==============================] - 10s 2ms/step - loss: 77.1105 - accuracy: 0.5042
Epoch 4/10
5488/5488 [==============================] - 10s 2ms/step - loss: 4.8525 - accuracy: 0.5308
Epoch 5/10
5488/5488 [==============================] - 10s 2ms/step - loss: 11.1169 - accuracy: 0.5314
Epoch 6/10
5488/5488 [==============================] - 10s 2ms/step - loss: 6.5719 - accuracy: 0.5319
Epoch 7/10
5488/5488 [==============================] - 10s 2ms/step - loss: 0.7884 - accuracy: 0.5320
Epoch 8/10
5488/5488 [==============================] - 10s 2ms/step - loss: 0.6912 - accuracy: 0.5320
Epoch 9/10
5488/5488 [==============================] - 10s 2ms/step - loss: 0.6912 - accuracy: 0.5320
Epoch 10/10
5488/5488 [==============================] - 10s 2ms/s

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


215/215 - 1s - loss: 0.5575 - accuracy: 0.7261 - 526ms/epoch - 2ms/step
Loss: 0.5574941635131836, Accuracy: 0.7260932922363281


In [23]:
# Create the model
nn_model = Sequential()
nn_model.add(Dense(32, input_shape=(50,), activation='relu'))
nn_model.add(Dense(16, activation='relu'))
nn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
nn_model.compile(optimizer='adam', loss='hinge', metrics=['accuracy'])

# Train the model
nn_model.fit(X_train_scaled, y_train, epochs=10, verbose=1)

Epoch 1/10
858/858 [==============================] - 4s 4ms/step - loss: 0.7768 - accuracy: 0.7147
Epoch 2/10
858/858 [==============================] - 3s 4ms/step - loss: 0.7383 - accuracy: 0.7313
Epoch 3/10
858/858 [==============================] - 1s 2ms/step - loss: 0.7367 - accuracy: 0.7318
Epoch 4/10
858/858 [==============================] - 2s 2ms/step - loss: 0.7361 - accuracy: 0.7326
Epoch 5/10
858/858 [==============================] - 2s 2ms/step - loss: 0.7355 - accuracy: 0.7329
Epoch 6/10
858/858 [==============================] - 1s 2ms/step - loss: 0.7354 - accuracy: 0.7329
Epoch 7/10
858/858 [==============================] - 1s 2ms/step - loss: 0.7354 - accuracy: 0.7327
Epoch 8/10
858/858 [==============================] - 1s 2ms/step - loss: 0.7351 - accuracy: 0.7331
Epoch 9/10
858/858 [==============================] - 2s 3ms/step - loss: 0.7354 - accuracy: 0.7327
Epoch 10/10
858/858 [==============================] - 2s 3ms/step - loss: 0.7351 - accuracy: 0.7330

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.7384 - accuracy: 0.7276 - 576ms/epoch - 3ms/step
Loss: 0.7384260296821594, Accuracy: 0.7275510430335999


In [25]:
# Create the model
nn_model = Sequential()
nn_model.add(Dense(32, input_shape=(50,), activation='relu'))
nn_model.add(Dense(16, activation='relu'))
nn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
nn_model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn_model.fit(X_train_scaled, y_train, epochs=10, verbose=1)

Epoch 1/10
858/858 [==============================] - 2s 2ms/step - loss: 0.5721 - accuracy: 0.7195
Epoch 2/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5557 - accuracy: 0.7281
Epoch 3/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5522 - accuracy: 0.7290
Epoch 4/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7308
Epoch 5/10
858/858 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7320
Epoch 6/10
858/858 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7317
Epoch 7/10
858/858 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7313
Epoch 8/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5480 - accuracy: 0.7318
Epoch 9/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5471 - accuracy: 0.7314
Epoch 10/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5473 - accuracy: 0.7323

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5582 - accuracy: 0.7299 - 546ms/epoch - 3ms/step
Loss: 0.5582093000411987, Accuracy: 0.7298833727836609


In [27]:
nn_model = Sequential()
nn_model.add(Dense(32, input_shape=(50,), activation='relu'))
nn_model.add(Dense(16, activation='relu'))
nn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=10, verbose=1)

Epoch 1/10
858/858 [==============================] - 3s 2ms/step - loss: 0.5804 - accuracy: 0.7145
Epoch 2/10
858/858 [==============================] - 2s 2ms/step - loss: 0.5546 - accuracy: 0.7283
Epoch 3/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5509 - accuracy: 0.7298
Epoch 4/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5488 - accuracy: 0.7306
Epoch 5/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5473 - accuracy: 0.7304
Epoch 6/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5466 - accuracy: 0.7320
Epoch 7/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5454 - accuracy: 0.7321
Epoch 8/10
858/858 [==============================] - 1s 2ms/step - loss: 0.5457 - accuracy: 0.7308
Epoch 9/10
858/858 [==============================] - 2s 2ms/step - loss: 0.5446 - accuracy: 0.7322
Epoch 10/10
858/858 [==============================] - 3s 3ms/step - loss: 0.5441 - accuracy: 0.7345

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5539 - accuracy: 0.7274 - 564ms/epoch - 3ms/step
Loss: 0.5539432168006897, Accuracy: 0.7274052500724792


In [29]:
# Save the model
nn_model.save("AlphabetSoupCharity_Optimization.h5")